In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats as s

# REGRESIÓN LOGISTICA

## QUEREMOS CREAR UN MODELO QUE NOS PERMITA DETERMINAR EL SEXO DE UN PINGUINO

In [6]:
penguins_df = sns.load_dataset('penguins')
penguins_df = penguins_df.dropna()
penguins_df

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,Male
...,...,...,...,...,...,...,...
338,Gentoo,Biscoe,47.2,13.7,214.0,4925.0,Female
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,Female
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,Male
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,Female


## AGREGAMOS VARIABLE NÚMERICA PARA EL SEXO

In [7]:
penguins_df = (
    penguins_df
    .assign(
        numeric_sex=lambda df: df.sex.replace(['Female','Male'],[0,1])
    )
)
penguins_df

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,numeric_sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male,1
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female,0
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female,0
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female,0
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,Male,1
...,...,...,...,...,...,...,...,...
338,Gentoo,Biscoe,47.2,13.7,214.0,4925.0,Female,0
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,Female,0
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,Male,1
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,Female,0


REGRESIÓN LOGISTICA

In [9]:
smf.logit(
    formula='numeric_sex ~ flipper_length_mm + bill_length_mm + bill_depth_mm + C(island)',
    data=penguins_df
).fit().summary()

Optimization terminated successfully.
         Current function value: 0.360900
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            numeric_sex   No. Observations:                  333
Model:                          Logit   Df Residuals:                      327
Method:                           MLE   Df Model:                            5
Date:                Tue, 27 Aug 2024   Pseudo R-squ.:                  0.4793
Time:                        00:58:32   Log-Likelihood:                -120.18
converged:                       True   LL-Null:                       -230.80
Covariance Type:            nonrobust   LLR p-value:                 8.021e-46
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                -61.4464      6.944     -8.849      0.000     -75.057     -47.836
C(island)[T.Dream]        -1.5596      0.493     -3.163      0.002      -2.526      -0.593
C(island)[T.Torgersen]    -1.0323      0.599     -1.725      0.085      -2.205       0.141
flipper_length_mm          0.1393      0.024      5.874      0.000       0.093       0.186
bill_length_mm             0.1413      0.045      3.150      0.002       0.053       0.229
bill_depth_mm              1.6401      0.185      8.864      0.000       1.277       2.003
==========================================================================================
"""

In [19]:
model_sex = smf.logit(
    formula='numeric_sex ~ flipper_length_mm + bill_length_mm + bill_depth_mm + C(island)',
    data=penguins_df
).fit()
model_sex.params

Optimization terminated successfully.
         Current function value: 0.360900
         Iterations 7


,0
Intercept,-61.446368
C(island)[T.Dream],-1.559644
C(island)[T.Torgersen],-1.032347
flipper_length_mm,0.139336
bill_length_mm,0.141288
bill_depth_mm,1.640066


In [20]:
is_male_predictions = pd.DataFrame(
    dict(
        actual_value = penguins_df.sex.replace(['Male', 'Female'], [1,0]),
        predicted_values = model_sex.predict().round()
    )
)
is_male_predictions

,actual_value,predicted_values
0,1,0.0
1,0,0.0
2,0,0.0
4,0,1.0
5,1,1.0
...,...,...
338,0,0.0
340,0,0.0
341,1,1.0
342,0,0.0


## ANALIZANDO VARIABLES CATEGORICAS

### ANALIZANDO LAS ESPECIES

In [21]:
(
    penguins_df
    .value_counts(['island','sex','species'])
    .reset_index(name='count')
)

,island,sex,species,count
0,Biscoe,Male,Gentoo,61
1,Biscoe,Female,Gentoo,58
2,Dream,Female,Chinstrap,34
3,Dream,Male,Chinstrap,34
4,Dream,Male,Adelie,28
5,Dream,Female,Adelie,27
6,Torgersen,Female,Adelie,24
7,Torgersen,Male,Adelie,23
8,Biscoe,Female,Adelie,22
9,Biscoe,Male,Adelie,22


In [22]:
penguins_df.species.unique()

array(['Adelie', 'Chinstrap', 'Gentoo'], dtype=object)

In [23]:
penguins_df = (
    penguins_df
    .assign(is_adelie=lambda df : df.species.replace(['Adelie','Chinstrap','Gentoo'],[1,0,0]))
)
penguins_df

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,numeric_sex,is_adelie
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male,1,1
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female,0,1
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female,0,1
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female,0,1
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,Male,1,1
...,...,...,...,...,...,...,...,...,...
338,Gentoo,Biscoe,47.2,13.7,214.0,4925.0,Female,0,0
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,Female,0,0
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,Male,1,0
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,Female,0,0


REGRESIÓN LOGISTICA PARA ESPECIES

In [24]:
model_is_adelie = smf.logit(
    formula='is_adelie ~ flipper_length_mm + C(sex)',
    data=penguins_df
).fit()
model_is_adelie.params

Optimization terminated successfully.
         Current function value: 0.355225
         Iterations 8


,0
Intercept,40.568368
C(sex)[T.Male],1.282656
flipper_length_mm,-0.209705


PREDICCIONES

In [ ]:
is_adelie_predictions = pd.DataFrame(
    dict(
        actual_value = penguins_df.is_adelie,
        predicted_values = model_is_adelie.predict().round()
    )
)
is_adelie_predictions

In [26]:
(
    is_adelie_predictions
    .value_counts(['actual_value','predicted_values'])
    .reset_index(name='count')
)

,actual_value,predicted_values,count
0,0,0.0,151
1,1,1.0,129
2,0,1.0,36
3,1,0.0,17
